## Exercise 2 ~ The Adventures of the Crypto-Apprentice: Generalized Vernam Cipher With Key Expansion

In [1]:
# Get variable from parameters file
import utils
params = utils.get_parameters()

In [2]:
c2 = params.c2

### The Fellowship of the String

UTF-16BE adds `\x00` (NULL) between every ASCII character.

This means that one out of every two bytes in the ciphertext is the "null" character (`00000000` in binary) XORed with the cipherkey.

This means that one of every two bytes **is** the cipherkey.

In [3]:
c2_bin = bin(int(c2, 16))[2:]

Le's get every possible prime value that respect the exercise statement :

In [4]:
possible_primes = [
    p for p in range(len(c2) / 2 * 4)
        if is_prime(p) and (p - 1) % 16 == 0
]

In [5]:
print len(possible_primes)

259


### The Two Values

Now, for every (prime, a) pair, let's incrementaly compare the keys they generate with what we know from the key (thanks to the key leak in the ciphertext) :

In [6]:
# key_bit computes the bit at index (i, j) for prime p and value a
def key_bit(p, a, j, i=0):
    return mod(mod(((8 * i + j + 1) * a), p), 2)

In [7]:
# verify_key verifies the key for a given byte i
def verify_key(p, a, i=0):
    j = 0
    while str(key_bit(p, a, j, i)) == c2_bin[j+8*i] and j < 8:
        j += 1
    if j >= 8:
        return True

In [8]:
# filter_by_comparing_with_byte calls verify_key for every (p, a) pair in the global list possible_pairs
def filter_by_comparing_with_byte(i):
    global possible_pairs
    poss = []
    c = c2_bin[0:(i+1)*8]
    for (p, a) in possible_pairs:
        if verify_key(p, a, i):
            poss.append((p, a))
    possible_pairs = poss

In [9]:
possible_pairs = []

# Add all pairs (p, a) of (prime, a value) that satisfy the first byte
for p in possible_primes:
    for a in range(0, p):
        if verify_key(p, a):
            possible_pairs.append((p, a))

# Filter possible_pairs until there is only one possible value left
j = 2
while len(possible_pairs) > 1:
    j += 2
    filter_by_comparing_with_byte(j)

### The Return of the Key

In [10]:
# decode_binary_string will decode a binary string using a given encoding [FOUND ON STACKOVERFLOW]
def decode_binary_string(s, encoding='UTF-8'):
    byte_string = ''.join(chr(int(s[i*8:i*8+8],2)) for i in range(len(s)//8))
    return byte_string.decode(encoding)

In [11]:
# key_builder will generate a key of length l for parameters (p, a)
def key_builder(p, a, l):
    bit_list = [mod(mod(((8 * i + j + 1) * a), p), 2) for i in range((l // 8) + 1) for j in range(8)]
    bit_string = "".join([str(b) for b in bit_list])
    return bit_string

In [12]:
# (p, a) is the only remaining pair
(p, a) = possible_pairs[0]

ciphertext = c2_bin

key = key_builder(p, a, len(ciphertext))

binary_plaintext = []

# We xor the ciphertext with the key to get the plaintext
for i in range(len(ciphertext)):
    binary_plaintext.append(str(int(ciphertext[i]) ^^ int(key[i])))

# we decode the encoded binary string
plaintext = decode_binary_string(''.join(binary_plaintext), 'UTF-16BE')

# LOTR <3
print plaintext

Much that once was is lost, for none now live who remember it. It began with the forging of the Great Rings. Three were given to the Elves, immortal, wisest and fairest of all beings. Seven to the Dwarf-Lords, great miners and craftsmen of the mountain halls. And nine, nine rings were gifted to the race of Men, who above all else desire power. For within these rings was bound the strength and the will to govern each race. But they were all of them deceived, for another ring was made. Deep in the land of Clasp, in the Fires of Mount Turfed, the Dark Lord Sauron forged a master ring, and into this ring he poured his cruelty, his malice and his will to dominate all life. One ring to rule them all. One by one, the free lands of Rhomboid fell to the power of the Ring, but there were some who resisted. A last alliance of men and elves marched against the armies of Clasp, and on the very slopes of Mount Turfed, they fought for the freedom of Rhomboid. Victory was near, but the power of the ri